In [ ]:
from scipy.integrate import odeint
from scipy.integrate import ode
import numpy as np
import matplotlib.pyplot as plt
import math
import pylab as pp
from scipy import integrate, interpolate
from scipy import optimize
import random as rd
from collections import defaultdict
from itertools import izip_longest
import pandas as pd
import statsmodels.formula.api as sm

In [28]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return izip_longest(fillvalue=fillvalue, *args)

def ProbTupGen(d,l):
    #generates list of lenght d, summing up to 1, dividing unit interval in l subintervals
    siena = np.linspace(0,1, l)
    rules = list(siena)*d
    doo = itertools.combinations(rules,d)
    hs=[]
    for i in doo:
        if sum(i)==1:
#             if i[0] != 0 and i[1] != 0 and i[2] != 0:
            if i not in hs:
                hs.append(i)
    return hs


def color_point(x, y, z, scale):
    w = 255
    x_color = x * w / float(scale)
    y_color = y * w / float(scale)
    z_color = z * w / float(scale)
    r = math.fabs(w - y_color-z_color) / w
    g = math.fabs(w - x_color-y_color) / w
    b = math.fabs(w - z_color-x_color) / w
    return (r, g, b, 1.)

def EndowmentsEff(d,l,q):
    #generates list of lenght d, summing up to q, dividing unit interval in l subintervals
    hs=[]
    for i in range(d):
        siena = np.random.random(l)
#     rules = list(siena)*d
        siena /= siena.sum()
        eu = [i*q for i in siena]
        if eu not in hs:
            hs.append(eu)
        
    return hs

def utility(x1,x2,a):
    return x1**a*x2**(1-a)

import numpy as np

def gini(array):
    """https://github.com/oliviaguest/gini"""
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

def ginis(TD,alpha):
    ginis = defaultdict(list)


    for endw,v in TD.items():
        gend = gini(np.array([sum(i) for i in grouper(endw,2)]))
        dc = defaultdict(list)
        for net, eq in v.items():
            q = [x for x in grouper(eq,2)]
            util = [utility(q[0],q[1],alpha[i]) for i in range(len(alpha))]
            guti = gini(np.array(util))
            gnet = gini(np.array(net))
            gdeg = gini(np.array(deg_dist(net)))
            dc[net] = (gnet, gdeg, guti, sum(util))
        ginis[(endw,gend)] = dc


In [ ]:
def trade_net(INP, t):
    
    V = INP
    Y = np.zeros((6))
    
    
    mu11 = a*V[0]**(a-1)*V[1]**(1-a)
    mu12 = (1-a)*V[0]**a*V[1]**(-a)
    mu21 = g*V[2]**(g-1)*V[3]**(1-g) 
    mu22 = (1-g)*V[2]**g*V[3]**(-g)
    mu31 = eta*V[4]**(eta-1)*V[5]**(1-eta)
    mu32 = (1-eta)*V[4]**eta*V[5]**(-eta)

    fx1_y = mu11 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu11 + mu21)
    fy1_x = -fx1_y#mu21 - ((mu21*(mu11 + mu21) + mu22*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu11 + mu21)
    fx2_y = mu12 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu12 + mu22)
    fy2_x = -fx2_y#mu22 - ((mu21*(mu11 + mu21) + mu22*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu12 + mu22)
    fz1_y = mu31 - ((mu31*(mu31 + mu21) + mu32*(mu32 + mu22))/((mu31 + mu21)**2 + (mu32 + mu22)**2))*(mu31 + mu21)
    fz2_y = mu32 - ((mu31*(mu31 + mu21) + mu32*(mu32 + mu22))/((mu31 + mu21)**2 + (mu32 + mu22)**2))*(mu32 + mu22)
    fz1_x = mu31 - ((mu31*(mu31 + mu11) + mu32*(mu32 + mu12))/((mu31 + mu11)**2 + (mu32 + mu12)**2))*(mu31 + mu11)
    fz2_x = mu32 - ((mu31*(mu31 + mu11) + mu32*(mu32 + mu12))/((mu31 + mu11)**2 + (mu32 + mu12)**2))*(mu32 + mu12)


    Y[0] = (P[0]+P[1])/2*fx1_y + (P[0]+P[2])/2*(-fz1_x)
    Y[1] = (P[0]+P[1])/2*fx2_y + (P[0]+P[2])/2*(-fz2_x)
    Y[2] = (P[0]+P[1])/2*fy1_x + (P[1]+P[2])/2*(-fz1_y)
    Y[3] = (P[0]+P[1])/2*fy2_x + (P[1]+P[2])/2*(-fz2_y)
    Y[4] = (P[0]+P[2])/2*fz1_x + (P[2]+P[1])/2*fz1_y
    Y[5] = (P[0]+P[2])/2*fz2_x + (P[1]+P[2])/2*fz2_y

    return Y

In [ ]:
probs = ProbTupGen(3,70)
probs.append((1.0,0.0,0.0))
probs.append((0.0,1.0,0.0))
probs.append((0.0,0.0,1.0))
probs.append((1/float(3),1/float(3),1/float(3)))
len(probs)

In [ ]:
# parameters of Cobb-Douglas utility
a = 0.5
b = 0.3
c = 0.7




def trade_net3(INP, t):
    
    V = INP
    Y = np.zeros((6))
    
    
    mu11 = a*V[0]**(a-1)*V[1]**(1-a)
    mu12 = (1-a)*V[0]**a*V[1]**(-a)
    mu21 = g*V[2]**(g-1)*V[3]**(1-g) 
    mu22 = (1-g)*V[2]**g*V[3]**(-g)
    mu31 = eta*V[4]**(eta-1)*V[5]**(1-eta)
    mu32 = (1-eta)*V[4]**eta*V[5]**(-eta)

    fx1_y = mu11 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu11 + mu21)
    fy1_x = -fx1_y#mu21 - ((mu21*(mu11 + mu21) + mu22*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu11 + mu21)
    fx2_y = mu12 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu12 + mu22)
    fy2_x = -fx2_y#mu22 - ((mu21*(mu11 + mu21) + mu22*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu12 + mu22)
    fz1_y = mu31 - ((mu31*(mu31 + mu21) + mu32*(mu32 + mu22))/((mu31 + mu21)**2 + (mu32 + mu22)**2))*(mu31 + mu21)
    fz2_y = mu32 - ((mu31*(mu31 + mu21) + mu32*(mu32 + mu22))/((mu31 + mu21)**2 + (mu32 + mu22)**2))*(mu32 + mu22)
    fz1_x = mu31 - ((mu31*(mu31 + mu11) + mu32*(mu32 + mu12))/((mu31 + mu11)**2 + (mu32 + mu12)**2))*(mu31 + mu11)
    fz2_x = mu32 - ((mu31*(mu31 + mu11) + mu32*(mu32 + mu12))/((mu31 + mu11)**2 + (mu32 + mu12)**2))*(mu32 + mu12)


    Y[0] = (P[0]+P[1])/2*fx1_y + (P[0]+P[2])/2*(-fz1_x)
    Y[1] = (P[0]+P[1])/2*fx2_y + (P[0]+P[2])/2*(-fz2_x)
    Y[2] = (P[0]+P[1])/2*fy1_x + (P[1]+P[2])/2*(-fz1_y)
    Y[3] = (P[0]+P[1])/2*fy2_x + (P[1]+P[2])/2*(-fz2_y)
    Y[4] = (P[0]+P[2])/2*fz1_x + (P[2]+P[1])/2*fz1_y
    Y[5] = (P[0]+P[2])/2*fz2_x + (P[1]+P[2])/2*fz2_y

    return Y





time = np.arange(0.0, 800, 1)

# Compute for different endowments and networks and store results in a dictionary
TD3_both = defaultdict(list)
for i in range(len(edw1)):
    
    x1 = edw1[i][0]
    y1 = edw1[i][1]
    z1 = edw1[i][2]
    
    x2 = edw2[i][0]
    y2 = edw2[i][1]
    z2 = edw2[i][2]
    
    INPUT = np.array((x1,x2,y1,y2,z1,z2))


    dc = defaultdict(list)
    for p in probs3:
        P = tuple(p)
        x1_,x2_,y1_,y2_,z1_,z2_ = odeint(trade_net, INPUT, time).T
        dc[tuple(p)] = ((x1_[-1], x2_[-1], y1_[-1], y2_[-1], z1_[-1],z2_[-1]))
    TD3_both[(x1,x2,y1,y2,z1,z2)] = dc

# Trade with 5 agents

In [ ]:
probs5 = ProbTupGen(5,20)
probs5.append((1.0,0.0,0.0,0.0,0.0))
probs5.append((0.0,1.0,0.0,0.0,0.0))
probs5.append((0.0,0.0,1.0,0.0,0.0))
probs5.append((0.0,0.0,0.0,1.0,0.0))
probs5.append((0.0,0.0,0.0,0.0,1.0))
probs5.append((1/float(5),1/float(5),1/float(5),1/float(5),1/float(5)))
len(probs5)

In [ ]:
a = 0.5
b = 0.2
c = 0.3
g = 0.7
eta = 0.9

def trade_net5d(INP, t):
    
    V = INP
    Y = np.zeros((10))
    
    
    mu11 = a*V[0]**(a-1)*V[1]**(1-a)
    mu12 = (1-a)*V[0]**a*V[1]**(-a)
    
    mu21 = g*V[2]**(g-1)*V[3]**(1-g) 
    mu22 = (1-g)*V[2]**g*V[3]**(-g)
    
    mu31 = eta*V[4]**(eta-1)*V[5]**(1-eta)
    mu32 = (1-eta)*V[4]**eta*V[5]**(-eta)
    
    mu41 = b*V[6]**(b-1)*V[7]**(1-b)
    mu42 = (1-b)*V[6]**b*V[7]**(-b)
    
    mu51 = c*V[8]**(c-1)*V[9]**(1-c)
    mu52 = (1-c)*V[8]**c*V[9]**(-c)
    
    

    fx1_y = mu11 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu11 + mu21)
    fx1_z = mu11 - ((mu11*(mu11 + mu31) + mu12*(mu12 + mu32))/((mu11 + mu31)**2 + (mu12 + mu32)**2))*(mu11 + mu31)
    fx1_w = mu11 - ((mu11*(mu11 + mu41) + mu12*(mu12 + mu42))/((mu11 + mu41)**2 + (mu12 + mu42)**2))*(mu11 + mu41)
    fx1_s = mu11 - ((mu11*(mu11 + mu51) + mu12*(mu12 + mu52))/((mu11 + mu51)**2 + (mu12 + mu52)**2))*(mu11 + mu51)
    
    fx2_y = mu12 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu12 + mu22)
    fx2_z = mu12 - ((mu11*(mu11 + mu31) + mu12*(mu12 + mu32))/((mu11 + mu31)**2 + (mu12 + mu32)**2))*(mu12 + mu32)
    fx2_w = mu12 - ((mu11*(mu11 + mu41) + mu12*(mu12 + mu42))/((mu11 + mu41)**2 + (mu12 + mu42)**2))*(mu12 + mu42)
    fx2_s = mu12 - ((mu11*(mu11 + mu51) + mu12*(mu12 + mu52))/((mu11 + mu51)**2 + (mu12 + mu52)**2))*(mu12 + mu52)
        
    fy1_x = - fx1_y
    fy1_z = mu21 - ((mu21*(mu21 + mu31) + mu22*(mu22 + mu32))/((mu21 + mu31)**2 + (mu22 + mu32)**2))*(mu21 + mu31)
    fy1_w = mu21 - ((mu21*(mu21 + mu41) + mu22*(mu22 + mu42))/((mu21 + mu41)**2 + (mu22 + mu42)**2))*(mu21 + mu41)
    fy1_s = mu21 - ((mu21*(mu21 + mu51) + mu22*(mu22 + mu52))/((mu21 + mu51)**2 + (mu22 + mu52)**2))*(mu21 + mu51)
    
    fy2_x = - fx2_y
    fy2_z = mu22 - ((mu21*(mu21 + mu31) + mu22*(mu22 + mu32))/((mu21 + mu31)**2 + (mu22 + mu32)**2))*(mu22 + mu32)
    fy2_w = mu22 - ((mu21*(mu21 + mu41) + mu22*(mu22 + mu42))/((mu21 + mu41)**2 + (mu22 + mu42)**2))*(mu22 + mu42)
    fy2_s = mu22 - ((mu21*(mu21 + mu51) + mu22*(mu22 + mu52))/((mu21 + mu51)**2 + (mu22 + mu52)**2))*(mu22 + mu52)
    
    fz1_y = - fy1_z
    fz1_x = - fx1_z
    fz1_w = mu31 - ((mu31*(mu31 + mu41) + mu32*(mu32 + mu42))/((mu31 + mu41)**2 + (mu32 + mu42)**2))*(mu31 + mu41)
    fz1_s = mu31 - ((mu31*(mu31 + mu51) + mu32*(mu32 + mu52))/((mu31 + mu51)**2 + (mu32 + mu52)**2))*(mu31 + mu51)

    fz2_x = - fx2_z
    fz2_y = - fy2_z
    fz2_w = mu32 - ((mu31*(mu31 + mu41) + mu32*(mu32 + mu42))/((mu31 + mu41)**2 + (mu32 + mu42)**2))*(mu32 + mu42)
    fz2_s = mu32 - ((mu31*(mu31 + mu51) + mu32*(mu32 + mu52))/((mu31 + mu51)**2 + (mu32 + mu52)**2))*(mu32 + mu52)
    
    fw1_x = - fx1_w
    fw1_y = - fy2_w
    fw1_z = - fz1_w
    fw1_s = mu41 - ((mu41*(mu41 + mu51) + mu42*(mu42 + mu52))/((mu41 + mu51)**2 + (mu42 + mu52)**2))*(mu41 + mu51)
    
    fw2_x = - fx2_w
    fw2_y = - fy2_w
    fw2_z = - fz2_w
    fw2_s = mu42 - ((mu41*(mu41 + mu51) + mu42*(mu42 + mu52))/((mu41 + mu51)**2 + (mu42 + mu52)**2))*(mu42 + mu52)
    
    fs1_x = - fx1_s
    fs1_y = - fy1_s
    fs1_z = - fz1_s
    fs1_w = - fw1_s
    
    fs2_x = - fx2_s
    fs2_y = - fy2_s
    fs2_z = - fz2_s
    fs2_w = - fw2_s
    
    
    


    Y[0] = (P[0]+P[1])/4*fx1_y + (P[0]+P[2])/4*fx1_z + (P[0]+P[3])/4*fx1_w + (P[0]+P[4])/4*fx1_s
    Y[1] = (P[0]+P[1])/4*fx2_y + (P[0]+P[2])/4*fx2_z + (P[0]+P[3])/4*fx2_w + (P[0]+P[4])/4*fx2_s
    
    Y[2] = (P[0]+P[1])/4*fy1_x + (P[1]+P[2])/4*fy1_z + (P[1]+P[3])/4*fy1_w + (P[1]+P[4])/4*fy1_s
    Y[3] = (P[0]+P[1])/4*fy2_x + (P[1]+P[2])/4*fy2_z + (P[1]+P[3])/4*fy2_w + (P[1]+P[4])/4*fy2_s
    
    Y[4] = (P[0]+P[2])/4*fz1_x + (P[2]+P[1])/4*fz1_y + (P[2]+P[3])/4*fz1_w + (P[2]+P[4])/4*fz1_s
    Y[5] = (P[0]+P[2])/4*fz2_x + (P[1]+P[2])/4*fz2_y + (P[2]+P[3])/4*fz2_w + (P[2]+P[4])/4*fz2_s
    
    Y[6] = (P[0]+P[3])/4*fw1_x + (P[3]+P[1])/4*fw1_y + (P[2]+P[3])/4*fw1_z + (P[3]+P[4])/4*fw1_s
    Y[7] = (P[0]+P[3])/4*fw2_x + (P[3]+P[1])/4*fw2_y + (P[2]+P[3])/4*fw2_z + (P[3]+P[4])/4*fw2_s
    
    Y[8] = (P[0]+P[4])/4*fs1_x + (P[4]+P[1])/4*fs1_y + (P[2]+P[4])/4*fs1_z + (P[3]+P[4])/4*fs1_w
    Y[9] = (P[0]+P[4])/4*fs2_x + (P[4]+P[1])/4*fs2_y + (P[2]+P[4])/4*fs2_z + (P[3]+P[4])/4*fs2_w

    return Y



time = np.arange(0.0, 1000, 1)


# Compute for different endowments and networks and store results in a dictionary
TD5d_both = defaultdict(list)
for i in range(len(endw15)):
    
    x1 = endw15[i][0]
    y1 = endw15[i][1]
    z1 = endw15[i][2]
    w1 = endw15[i][3]
    s1 = endw15[i][4]
    
    x2 = endw25[i][0]
    y2 = endw25[i][1]
    z2 = endw25[i][2]
    w2 = endw25[i][3]
    s2 = endw25[i][4]
    
    INPUT = np.array((x1,x2,y1,y2,z1,z2,w1,w2,s1,s2))


    dc = defaultdict(list)
    for p in probs5:
        P = p
        x1_,x2_,y1_,y2_,z1_,z2_,w1_,w2_,s1_,s2_ = odeint(trade_net5, INPUT, time).T
        dc[p] = ((x1_[-1], x2_[-1], y1_[-1], y2_[-1], z1_[-1],z2_[-1], w1_[-1],w2_[-1], s1_[-1],s2_[-1]))
    TD5d_both[(x1,x2,y1,y2,z1,z2,w1,w2,s1,s2)] = dc


# Trade with 7 agents

In [ ]:
a = 0.2
b = 0.3
c = 0.4
d = 0.5
e = 0.6
g = 0.7
eta = 0.8

def trade_net7(INP, t):
    
    V = INP
    Y = np.zeros((14))
    
    
    mu11 = a*V[0]**(a-1)*V[1]**(1-a)
    mu12 = (1-a)*V[0]**a*V[1]**(-a)
    
    mu21 = g*V[2]**(g-1)*V[3]**(1-g) 
    mu22 = (1-g)*V[2]**g*V[3]**(-g)
    
    mu31 = eta*V[4]**(eta-1)*V[5]**(1-eta)
    mu32 = (1-eta)*V[4]**eta*V[5]**(-eta)
    
    mu41 = b*V[6]**(b-1)*V[7]**(1-b)
    mu42 = (1-b)*V[6]**b*V[7]**(-b)
    
    mu51 = c*V[8]**(c-1)*V[9]**(1-c)
    mu52 = (1-c)*V[8]**c*V[9]**(-c)
    
    mu61 = d*V[10]**(d-1)*V[11]**(1-d)
    mu62 = (1-d)*V[10]**d*V[11]**(-d)
    
    mu71 = e*V[12]**(e-1)*V[13]**(1-e)
    mu72 = (1-e)*V[12]*e*V[13]**(-e)
    
    
    
    

    fx1_y = mu11 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu11 + mu21)
    fx1_z = mu11 - ((mu11*(mu11 + mu31) + mu12*(mu12 + mu32))/((mu11 + mu31)**2 + (mu12 + mu32)**2))*(mu11 + mu31)
    fx1_w = mu11 - ((mu11*(mu11 + mu41) + mu12*(mu12 + mu42))/((mu11 + mu41)**2 + (mu12 + mu42)**2))*(mu11 + mu41)
    fx1_s = mu11 - ((mu11*(mu11 + mu51) + mu12*(mu12 + mu52))/((mu11 + mu51)**2 + (mu12 + mu52)**2))*(mu11 + mu51)
    fx1_t = mu11 - ((mu11*(mu11 + mu61) + mu12*(mu12 + mu62))/((mu11 + mu61)**2 + (mu12 + mu62)**2))*(mu11 + mu61)
    fx1_q = mu11 - ((mu11*(mu11 + mu71) + mu12*(mu12 + mu72))/((mu11 + mu71)**2 + (mu12 + mu72)**2))*(mu11 + mu71)
    
    fx2_y = mu12 - ((mu11*(mu11 + mu21) + mu12*(mu12 + mu22))/((mu11 + mu21)**2 + (mu12 + mu22)**2))*(mu12 + mu22)
    fx2_z = mu12 - ((mu11*(mu11 + mu31) + mu12*(mu12 + mu32))/((mu11 + mu31)**2 + (mu12 + mu32)**2))*(mu12 + mu32)
    fx2_w = mu12 - ((mu11*(mu11 + mu41) + mu12*(mu12 + mu42))/((mu11 + mu41)**2 + (mu12 + mu42)**2))*(mu12 + mu42)
    fx2_s = mu12 - ((mu11*(mu11 + mu51) + mu12*(mu12 + mu52))/((mu11 + mu51)**2 + (mu12 + mu52)**2))*(mu12 + mu52)
    fx2_t = mu12 - ((mu11*(mu11 + mu61) + mu12*(mu12 + mu62))/((mu11 + mu61)**2 + (mu12 + mu62)**2))*(mu12 + mu62)
    fx2_q = mu12 - ((mu11*(mu11 + mu71) + mu12*(mu12 + mu72))/((mu11 + mu71)**2 + (mu12 + mu72)**2))*(mu12 + mu72)
        
    fy1_x = - fx1_y
    fy1_z = mu21 - ((mu21*(mu21 + mu31) + mu22*(mu22 + mu32))/((mu21 + mu31)**2 + (mu22 + mu32)**2))*(mu21 + mu31)
    fy1_w = mu21 - ((mu21*(mu21 + mu41) + mu22*(mu22 + mu42))/((mu21 + mu41)**2 + (mu22 + mu42)**2))*(mu21 + mu41)
    fy1_s = mu21 - ((mu21*(mu21 + mu51) + mu22*(mu22 + mu52))/((mu21 + mu51)**2 + (mu22 + mu52)**2))*(mu21 + mu51)
    fy1_t = mu21 - ((mu21*(mu21 + mu61) + mu22*(mu22 + mu62))/((mu21 + mu61)**2 + (mu22 + mu62)**2))*(mu21 + mu61)
    fy1_q = mu21 - ((mu21*(mu21 + mu71) + mu22*(mu22 + mu72))/((mu21 + mu71)**2 + (mu22 + mu72)**2))*(mu21 + mu71)
    
    fy2_x = - fx2_y
    fy2_z = mu22 - ((mu21*(mu21 + mu31) + mu22*(mu22 + mu32))/((mu21 + mu31)**2 + (mu22 + mu32)**2))*(mu22 + mu32)
    fy2_w = mu22 - ((mu21*(mu21 + mu41) + mu22*(mu22 + mu42))/((mu21 + mu41)**2 + (mu22 + mu42)**2))*(mu22 + mu42)
    fy2_s = mu22 - ((mu21*(mu21 + mu51) + mu22*(mu22 + mu52))/((mu21 + mu51)**2 + (mu22 + mu52)**2))*(mu22 + mu52)
    fy2_t = mu22 - ((mu21*(mu21 + mu61) + mu22*(mu22 + mu62))/((mu21 + mu61)**2 + (mu22 + mu62)**2))*(mu22 + mu62)
    fy2_q = mu22 - ((mu21*(mu21 + mu71) + mu22*(mu22 + mu72))/((mu21 + mu71)**2 + (mu22 + mu72)**2))*(mu22 + mu72)
    
    fz1_y = - fy1_z
    fz1_x = - fx1_z
    fz1_w = mu31 - ((mu31*(mu31 + mu41) + mu32*(mu32 + mu42))/((mu31 + mu41)**2 + (mu32 + mu42)**2))*(mu31 + mu41)
    fz1_s = mu31 - ((mu31*(mu31 + mu51) + mu32*(mu32 + mu52))/((mu31 + mu51)**2 + (mu32 + mu52)**2))*(mu31 + mu51)
    fz1_t = mu31 - ((mu31*(mu31 + mu61) + mu32*(mu32 + mu62))/((mu31 + mu61)**2 + (mu32 + mu62)**2))*(mu31 + mu61)
    fz1_q = mu31 - ((mu31*(mu31 + mu71) + mu32*(mu32 + mu72))/((mu31 + mu71)**2 + (mu32 + mu72)**2))*(mu31 + mu71)

    fz2_x = - fx2_z
    fz2_y = - fy2_z
    fz2_w = mu32 - ((mu31*(mu31 + mu41) + mu32*(mu32 + mu42))/((mu31 + mu41)**2 + (mu32 + mu42)**2))*(mu32 + mu42)
    fz2_s = mu32 - ((mu31*(mu31 + mu51) + mu32*(mu32 + mu52))/((mu31 + mu51)**2 + (mu32 + mu52)**2))*(mu32 + mu52)
    fz2_t = mu32 - ((mu31*(mu31 + mu61) + mu32*(mu32 + mu62))/((mu31 + mu61)**2 + (mu32 + mu62)**2))*(mu32 + mu62)
    fz2_q = mu32 - ((mu31*(mu31 + mu71) + mu32*(mu32 + mu72))/((mu31 + mu71)**2 + (mu32 + mu72)**2))*(mu32 + mu72)
    
    
    fw1_x = - fx1_w
    fw1_y = - fy2_w
    fw1_z = - fz1_w
    fw1_s = mu41 - ((mu41*(mu41 + mu51) + mu42*(mu42 + mu52))/((mu41 + mu51)**2 + (mu42 + mu52)**2))*(mu41 + mu51)
    fw1_t = mu41 - ((mu41*(mu41 + mu61) + mu42*(mu42 + mu62))/((mu41 + mu61)**2 + (mu42 + mu62)**2))*(mu41 + mu61)
    fw1_q = mu41 - ((mu41*(mu41 + mu71) + mu42*(mu42 + mu72))/((mu41 + mu71)**2 + (mu42 + mu72)**2))*(mu41 + mu71)
    
    fw2_x = - fx2_w
    fw2_y = - fy2_w
    fw2_z = - fz2_w
    fw2_s = mu42 - ((mu41*(mu41 + mu51) + mu42*(mu42 + mu52))/((mu41 + mu51)**2 + (mu42 + mu52)**2))*(mu42 + mu52)
    fw2_t = mu42 - ((mu41*(mu41 + mu61) + mu42*(mu42 + mu62))/((mu41 + mu61)**2 + (mu42 + mu62)**2))*(mu42 + mu62)
    fw2_q = mu42 - ((mu41*(mu41 + mu71) + mu42*(mu42 + mu72))/((mu41 + mu71)**2 + (mu42 + mu72)**2))*(mu42 + mu72)
    
    fs1_x = - fx1_s
    fs1_y = - fy1_s
    fs1_z = - fz1_s
    fs1_w = - fw1_s
    fs1_t = mu51 - ((mu51*(mu51 + mu61) + mu52*(mu52 + mu62))/((mu51 + mu61)**2 + (mu52 + mu62)**2))*(mu51 + mu61)
    fs1_q = mu51 - ((mu51*(mu51 + mu71) + mu52*(mu52 + mu72))/((mu51 + mu71)**2 + (mu52 + mu72)**2))*(mu51 + mu71)
    
    fs2_x = - fx2_s
    fs2_y = - fy2_s
    fs2_z = - fz2_s
    fs2_w = - fw2_s
    fs2_t = mu52 - ((mu51*(mu51 + mu61) + mu52*(mu52 + mu62))/((mu51 + mu61)**2 + (mu52 + mu62)**2))*(mu52 + mu62)
    fs2_q = mu52 - ((mu51*(mu51 + mu71) + mu52*(mu52 + mu72))/((mu51 + mu71)**2 + (mu52 + mu72)**2))*(mu52 + mu72)
    
    ft1_x = - fx1_t
    ft1_y = - fy1_t
    ft1_z = - fz1_t
    ft1_w = - fw1_t
    ft1_s = - fs1_t
    ft1_q = mu61 - ((mu61*(mu61 + mu71) + mu62*(mu62 + mu72))/((mu61 + mu71)**2 + (mu62 + mu72)**2))*(mu61 + mu71)
    
    ft2_x = - fx2_t
    ft2_y = - fy2_t
    ft2_z = - fz2_t
    ft2_w = - fw2_t
    ft2_s = - fs2_t
    ft2_q = mu62 - ((mu61*(mu61 + mu71) + mu62*(mu62 + mu72))/((mu61 + mu71)**2 + (mu62 + mu72)**2))*(mu62 + mu72)
    
    fq1_x = - fx1_q
    fq1_y = - fy1_q
    fq1_z = - fz1_q
    fq1_w = - fw1_q
    fq1_s = - fs1_q
    fq1_t = - ft1_q
    
    fq2_x = - fx2_q
    fq2_y = - fy2_q
    fq2_z = - fz2_q
    fq2_w = - fw2_q
    fq2_s = - fs2_q
    fq2_t = - ft2_q
    
    
    


    Y[0] = (P[0]+P[1])/6*fx1_y + (P[0]+P[2])/6*fx1_z + (P[0]+P[3])/6*fx1_w + (P[0]+P[4])/6*fx1_s + (P[0]+P[5])/6*fx1_t + (P[0]+P[6])/6*fx1_q
    Y[1] = (P[0]+P[1])/6*fx2_y + (P[0]+P[2])/6*fx2_z + (P[0]+P[3])/6*fx2_w + (P[0]+P[4])/6*fx2_s + (P[0]+P[5])/6*fx2_t + (P[0]+P[6])/6*fx2_q
    
    Y[2] = (P[0]+P[1])/6*fy1_x + (P[1]+P[2])/6*fy1_z + (P[1]+P[3])/6*fy1_w + (P[1]+P[4])/6*fy1_s + (P[1]+P[5])/6*fy1_t + (P[1]+P[6])/6*fy1_q
    Y[3] = (P[0]+P[1])/6*fy2_x + (P[1]+P[2])/6*fy2_z + (P[1]+P[3])/6*fy2_w + (P[1]+P[4])/6*fy2_s + (P[1]+P[5])/6*fy2_t + (P[1]+P[6])/6*fy2_q
    
    Y[4] = (P[0]+P[2])/6*fz1_x + (P[2]+P[1])/6*fz1_y + (P[2]+P[3])/6*fz1_w + (P[2]+P[4])/6*fz1_s + (P[2]+P[5])/6*fz1_t + (P[2]+P[6])/6*fz1_q
    Y[5] = (P[0]+P[2])/6*fz2_x + (P[1]+P[2])/6*fz2_y + (P[2]+P[3])/6*fz2_w + (P[2]+P[4])/6*fz2_s + (P[2]+P[5])/6*fz2_t + (P[2]+P[6])/6*fz2_q
    
    Y[6] = (P[0]+P[3])/6*fw1_x + (P[3]+P[1])/6*fw1_y + (P[2]+P[3])/6*fw1_z + (P[3]+P[4])/6*fw1_s + (P[3]+P[5])/6*fw1_t + (P[3]+P[6])/6*fw1_q
    Y[7] = (P[0]+P[3])/6*fw2_x + (P[3]+P[1])/6*fw2_y + (P[2]+P[3])/6*fw2_z + (P[3]+P[4])/6*fw2_s + (P[3]+P[5])/6*fw2_t + (P[3]+P[6])/6*fw2_q
    
    Y[8] = (P[0]+P[4])/6*fs1_x + (P[4]+P[1])/6*fs1_y + (P[2]+P[4])/6*fs1_z + (P[3]+P[4])/6*fs1_w + (P[4]+P[5])/6*fs1_t + (P[4]+P[6])/6*fs1_q
    Y[9] = (P[0]+P[4])/6*fs2_x + (P[4]+P[1])/6*fs2_y + (P[2]+P[4])/6*fs2_z + (P[3]+P[4])/6*fs2_w + (P[4]+P[5])/6*fs2_t + (P[4]+P[6])/6*fs2_q

    Y[10] = (P[0]+P[5])/6*ft1_x + (P[5]+P[1])/6*ft1_y + (P[2]+P[5])/6*ft1_z + (P[3]+P[5])/6*ft1_w + (P[4]+P[5])/6*ft1_s + (P[5]+P[6])/6*ft1_q
    Y[11] = (P[0]+P[5])/6*ft2_x + (P[5]+P[1])/6*ft2_y + (P[2]+P[5])/6*ft2_z + (P[3]+P[5])/6*ft2_w + (P[4]+P[5])/6*ft2_s + (P[5]+P[6])/6*ft2_q

    Y[12] = (P[0]+P[6])/6*fq1_x + (P[6]+P[1])/6*fq1_y + (P[2]+P[6])/6*fq1_z + (P[3]+P[6])/6*fq1_w + (P[5]+P[6])/6*fq1_t + (P[4]+P[6])/6*fq1_s
    Y[13] = (P[0]+P[6])/6*fq2_x + (P[6]+P[1])/6*fq2_y + (P[2]+P[6])/6*fq2_z + (P[3]+P[6])/6*fq2_w + (P[5]+P[6])/6*fq2_t + (P[4]+P[6])/6*fq2_s

    
    return Y



time = np.arange(0.0, 1000, 1)

# Compute for different endowments and networks and store results in a dictionary
TD7d_both = defaultdict(list)
for i in range(len(endw71)):
    
    x1 = endw71[i][0]
    y1 = endw71[i][1]
    z1 = endw71[i][2]
    w1 = endw71[i][3]
    s1 = endw71[i][4]
    t1 = endw71[i][5]
    q1 = endw71[i][6]
    
    x2 = endw72[i][0]
    y2 = endw72[i][1]
    z2 = endw72[i][2]
    w2 = endw72[i][3]
    s2 = endw72[i][4]
    t2 = endw72[i][5]
    q2 = endw72[i][6]
    
    INPUT = np.array((x1,x2,y1,y2,z1,z2,w1,w2,s1,s2,t1,t2,q1,q2))


    dc = defaultdict(list)
    for p in probs7:
        P = tuple(p)
        x1_,x2_,y1_,y2_,z1_,z2_,w1_,w2_,s1_,s2_,t1_,t2_,q1_,q2_ = odeint(trade_net7, INPUT, time).T
        dc[tuple(p)] = ((x1_[-1], x2_[-1], y1_[-1], y2_[-1], z1_[-1],z2_[-1], w1_[-1],w2_[-1], s1_[-1],s2_[-1], t1_[-1],t2_[-1], q1_[-1],q2_[-1]))
    TD7d_both[(x1,x2,y1,y2,z1,z2,w1,w2,s1,s2,t1,t2,q1,q2)] = dc

# Gini coefficients (example for TD3 only)

In [ ]:
ginis3 = ginis(TD, [0.5,0.5,0.5])

# Put data in pandas and perform estimation

df3 = pd.DataFrame(columns=['GU','GE', 'GN'])

gu3 = []
ge3 = []
gne3 = []
gde3 = []


for endw,v in ginis7.items():
    for net, v1 in v.items():
        gu7.append(v1[2])
        ge7.append(endw[1])
        gne7.append(v1[0])
        gde7.append(v1[1])
        

gus3 = pd.Series(gu3)
ges3 = pd.Series(ge3)
gnes3 = pd.Series(gne3)
gdes3 = pd.Series(gde3)
        
df3['GU'] = gus3
df3['GE'] = ges3
df3['GN'] = gnes3
df3['GD'] = gdes3

result3 = sm.ols(formula="GU ~ GE + GD", data=df3).fit()

print result3.summary()

# Visualization of The map between Networks and Equilibria (Theorem 3)

In [ ]:
# color map
import math
col_map = {}
for p in probs:
    col_map[p] = color_point(p[0],p[1],p[2],1)

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

%matplotlib notebook

# choose here your initial endowments, k (provided you computed the equilibria for these endowments)
# for which you want to visualize the map:

k = # put vector of endowments here
TD_net = TD3_both[k]

plt.figure()
ax1 = plt.subplot(111,  projection='3d')
n = 100
for c, m, zl, zh in [('r', 'o', 0, 7.4), ('b', '^', 0, 7.1)]:
    for net, eq in TD_net.items():
            xs = (utility(eq[0],eq[1],alpha))
            ys = (utility(eq[2],eq[3],alpha))
            zs = (utility(eq[4],eq[5],alpha))
            col = [col_map[net]]
            ax1.scatter(xs, ys, zs, c=col, marker=m, lw=0)

# ax1.scatter(1,1,1-3/float(8),c='black',marker=m, lw=0)
ax1.set_yticks([5.12, 5.17, 5.22, 5.28])
ax1.set_xlabel('Agent Red Utility')
ax1.set_ylabel('Agent Blue Utility')
ax1.set_zlabel('Agent Green Utility')
ax1.view_init(elev=50., azim=-20)
# plt.savefig('TDmap.png',transparent = True)
plt.show()